In [10]:
from bs4 import BeautifulSoup
import requests

uri  = "https://iopscience.iop.org/nsearch?terms=ZnTe&nextPage=2&previousPage=-1&currentPage=1&searchDatePeriod=anytime&orderBy=relevance&pageLength=50"

r1 = requests.get(uri)
coverpage = r1.content

In [11]:
soup1 = BeautifulSoup(coverpage, 'lxml')

In [28]:
td = soup1.find_all('div', {"class" : "art-list-item-body"})

In [29]:
len(td)

50

In [91]:
articulos = []
for i in td:
    a  = i.findChild('div', {"class" : "article-text view-text-small"}).findChild("p")

    b  = i.findChild('h2',  {"class" : "art-list-item-title"}).findChild("a")

    articulos.append([a.text ,b.text])

In [96]:
articulos[7][0]

'Excitation power and temperature-dependent photoluminescence (PL) spectra of the ZnTe epilayer grown on (100) GaAs substrate and ZnTe bulk crystal are investigated. The measurement results show that both the structures are of good structural quality due to their sharp bound excitonic emissions and absence of the deep level structural defect-related emissions. Furthermore, in contrast to the ZnTe bulk crystal, although excitonic emissions for the ZnTe epilayer are somewhat weak, perhaps due to As atoms diffusing from the GaAs substrate into the ZnTe epilayer and/or because of the strain-induced degradation of the crystalline quality of the ZnTe epilayer, neither the donor–acceptor pair (DAP) nor conduction band-acceptor (e–A) emissions are observed in the ZnTe epilayer. This indicates that by further optimizing the growth process it is possible to obtain a high-crystalline quality ZnTe heteroepitaxial layer that is comparable to the ZnTe bulk crystal.'

In [49]:
nsfw = []
for art in articulos:
    nsfw.append(str(art.find_all("p")))